In [1]:
import urllib.request
import gzip
import json
import sqlite3
from tqdm import tqdm

In [2]:
base_url = "https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2020-03-01/"
response = urllib.request.urlopen(base_url + "manifest.txt")
manifest = response.read().decode('utf-8')
files = manifest.split("\n")
files = files[:-2] # last two files are sample and readme

In [3]:
conn = sqlite3.connect("ontovec.db")
c = conn.cursor()
c.execute('''CREATE TABLE papers (id text, title text, abstract text, doi text)''')

In [4]:
def insertPaper(cursor, paper):
    sql = "INSERT INTO papers (id,title,abstract,doi) VALUES (?, ?, ?, ?)"
    cursor.execute(sql, (paper['id'], paper['title'], paper['paperAbstract'], paper['doi']))

In [8]:

for file in files:
    print(file)
    response = urllib.request.urlopen(base_url + file)
    unzippedBytes = gzip.decompress(response.read())
    decodedFile = unzippedBytes.decode('utf-8')
    jsons = decodedFile.split("\n")
    with tqdm(total=len(jsons)) as pbar:
        for jsonPaper in jsons:
            if jsonPaper and jsonPaper.strip():
                try:
                    paper = json.loads(jsonPaper)
                    insertPaper(c, paper)
                except:
                    print("failed: {}".format(jsonPaper))
            pbar.update(1)

s2-corpus-000.gz


100%|██████████| 999674/999674 [00:19<00:00, 51665.19it/s]


failed: 
s2-corpus-001.gz


100%|██████████| 999676/999676 [00:18<00:00, 55239.15it/s]


failed: 
s2-corpus-002.gz


KeyboardInterrupt: 

In [ ]:
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()